# **ДЗ NN3 - Создание RAG-системы на основе книги или кода с помощью LangChain**

#### Цель:
- Реализовать полностью локальный RAG-пайплайн.
- Генерация ответов и эмбеддинги выполняются через Ollama.
- Источник знаний — повесть И. С. Тургенева «Му-му» (в формате TXT или PDF).

In [1]:
import os
import re
from dataclasses import dataclass
from pathlib import Path
from textwrap import shorten

In [2]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

### Конфигурация

In [4]:
@dataclass
class Config:
    TXT_PATH: str = "data/mumu.txt"
    PDF_PATH: str = "data/mumu.pdf"
    FALLBACK_PATH: str = "data/mumu_excerpt.txt"

    # Backends
    LLM_BACKEND: str = "ollama"
    EMBED_BACKEND: str = "ollama"

    # Модели Ollama
    OLLAMA_MODEL: str = "qwen3:8b"
    OLLAMA_EMBED_MODEL: str = "nomic-embed-text"

    # Параметры чанков
    CHUNK_SIZE: int = 700
    CHUNK_OVERLAP: int = 120

    # Настройки ретривера
    TOP_K: int = 4


CFG = Config()

### Fallback-отрывок (чтобы запускался без книги)

In [5]:
def ensure_fallback(cfg: Config):
    Path("data").mkdir(exist_ok=True)
    if not os.path.exists(cfg.FALLBACK_PATH):
        excerpt = (
            "Иван Сергеевич Тургенев — «Му-му» (отрывок)\n\n"
            "Герасим — глухонемой дворник при барыне в Москве. Он подобрал щенка и назвал её Му-му.\n"
            "Он привязался к собаке, но барыне не понравился лай. Она велела избавиться от собаки.\n"
            "Герасим, мучаясь, исполнил приказ и вернулся в деревню.\n"
        )
        Path(cfg.FALLBACK_PATH).write_text(excerpt, encoding="utf-8")

### Загрузка документов

In [8]:
def load_documents(cfg: Config):
    if os.path.exists(cfg.TXT_PATH):
        print(f"TXT: {cfg.TXT_PATH}")
        loader = TextLoader(cfg.TXT_PATH, autodetect_encoding=True)
        docs = loader.load()
    elif os.path.exists(cfg.PDF_PATH):
        print(f"PDF: {cfg.PDF_PATH}")
        loader = PyPDFLoader(cfg.PDF_PATH)
        docs = loader.load()
    else:
        print(f"Книга не найдена. Использую отрывок: {cfg.FALLBACK_PATH}")
        loader = TextLoader(cfg.FALLBACK_PATH, autodetect_encoding=True)
        docs = loader.load()

    print("Документов:", len(docs))
    return docs

### Сплит на чанки

In [9]:
def split_documents(docs, cfg: Config):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=cfg.CHUNK_SIZE,
        chunk_overlap=cfg.CHUNK_OVERLAP,
        separators=["\n\n", "\n", " ", ""],
    )
    chunks = splitter.split_documents(docs)
    print(f"Чанков: {len(chunks)} | size={cfg.CHUNK_SIZE} overlap={cfg.CHUNK_OVERLAP}")
    return chunks

### Эмбеддинги + FAISS

In [10]:
def build_embeddings(cfg: Config):
    print(f" Embeddings via Ollama → {cfg.OLLAMA_EMBED_MODEL}")
    return OllamaEmbeddings(model=cfg.OLLAMA_EMBED_MODEL)


def build_retriever(chunks, embeddings, cfg: Config):
    vectordb = FAISS.from_documents(chunks, embeddings)
    retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": cfg.TOP_K})
    print(f"FAISS готов. TOP_K={cfg.TOP_K}")
    return retriever

### LLM + RetrievalQA c анти-галлюцинационной инструкцией

In [11]:
def build_llm(cfg: Config):
    print(f"LLM via Ollama → {cfg.OLLAMA_MODEL}")
    return OllamaLLM(model=cfg.OLLAMA_MODEL, temperature=0.2)


QA_TEMPLATE = """Ты — помощник-литературовед. Отвечай кратко и только по приведённым фрагментам. Если прямого ответа в контексте нет — так и скажи. Не показывай рассуждения, не добавляй теги <think>.

Вопрос: {question}

Контекст:
{context}

Ответ:"""

PROMPT = PromptTemplate(template=QA_TEMPLATE, input_variables=["question", "context"])


def make_qa_chain(llm, retriever):
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT},
    )
    return chain

### Очистка ответа

In [12]:
def clean_answer(text: str) -> str:
    # убрать блоки <think>…</think>
    text = re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL)
    # убрать **жирный** из Markdown
    text = re.sub(r"\*+", "", text)
    return text.strip()

### Вспомогательная ask() с выводом источников

In [15]:
def ask(qa_chain, question: str, max_len: int = 240):
    res = qa_chain.invoke({"query": question})  # используем invoke вместо __call__
    raw = res.get("result", "<нет ответа>")
    answer = clean_answer(raw)
    sources = res.get("source_documents", [])

    print("\n Вопрос:", question)
    print("\n Ответ:", answer)
    print("\n Источники:")
    for i, d in enumerate(sources, 1):
        meta = d.metadata if isinstance(d.metadata, dict) else {}
        loc = []
        if "source" in meta:
            loc.append(str(meta["source"]))
        if "page" in meta:
            loc.append(f"page {meta['page']}")
        loc_str = " | ".join(loc) if loc else "(метаданные отсутствуют)"
        print(f"[{i}] {loc_str}\n{shorten(d.page_content.strip(), max_len)}\n")

    return {"answer": answer, "sources": sources}

### main(): сборка пайплайна и 3 демо-запроса

In [20]:
def main():
    ensure_fallback(CFG)
    docs = load_documents(CFG)
    chunks = split_documents(docs, CFG)
    embeddings = build_embeddings(CFG)
    retriever = build_retriever(chunks, embeddings, CFG)
    llm = build_llm(CFG)
    qa_chain = make_qa_chain(llm, retriever)

    # Пример 1 — факт
    ask(qa_chain, "Как зовут немого героя повести (дворника/плотника)?")

    # Пример 2 — синтез
    ask(qa_chain, "Какие социальные темы поднимает Тургенев в «Му-му», и как эпизод с барыней их проявляет?")

    # Пример 3 — негативный кейс
    ask(qa_chain, "Есть ли в тексте прямое свидетельство, что барыня умерла в конце повести?")

if __name__ == "__main__":
    main()

TXT: data/mumu.txt
Документов: 1
Чанков: 110 | size=700 overlap=120
 Embeddings via Ollama → nomic-embed-text
FAISS готов. TOP_K=4
🗣️ LLM via Ollama → qwen3:8b

 Вопрос: Как зовут немого героя повести (дворника/плотника)?

 Ответ: Герасим

 Источники:
[1] data/mumu.txt
ее за полтинник, с тем только, чтоб он по крайней мере неделю продержал ее на привязи, и тотчас вернулся; но, не доезжая до дому, слез с извозчика и, обойдя двор кругом, с заднего переулка, через забор перескочил на двор: в калитку-то [...]

[2] data/mumu.txt
Весь следующий день Герасим не показывался, так что вместо его за водой должен был съездить кучер Потап, чем кучер Потап очень остался недоволен. Барыня спросила Гаврилу, исполнено ли ее приказание. Гаврило отвечал, что исполнено. На [...]

[3] data/mumu.txt
Герасим не изменил своего поведения ни в чем; только с реки он приехал без воды: он как-то на дороге разбил бочку; а на ночь в конюшне он так усердно чистил и тер свою лошадь, что та шаталась, как былинка на вет

### Пример 1. Поиск факта

**Вопрос**: *Как зовут немого героя повести (дворника/плотника)?*

**Ответ системы**: *Герасим*

**Анализ**:
Этот пример показывает, что система корректно решает задачу поиска конкретного факта. Имя главного героя встречается в тексте многократно, и ретривер безошибочно извлекает нужные чанки. Ответ совпадает с оригинальным текстом и демонстрирует точность работы RAG при прямых запросах. Это подтверждает, что пайплайн правильно загружает данные, делит их на части и достаёт релевантный фрагмент.

### Пример 2. Синтез информации

**Вопрос**: *Какие социальные темы поднимает Тургенев в «Му-му», и как эпизод с барыней их проявляет?*

**Ответ системы**: *Тургенев поднимает темы жестокости, несправедливости и неволи крестьян. Эпизод с барыней проявляет её безразличие к страданиям Муму, подчеркивая социальное неравенство и жестокость барства.*

**Анализ**:
Здесь задача сложнее: нужно было обобщить информацию из нескольких эпизодов. Ответ системы получился содержательным: она выделила основные социальные мотивы («жестокость», «несправедливость», «невольное положение крестьян») и связала их с поведением барыни. Это верная интерпретация, которая соответствует критическому прочтению текста. Таким образом, система показала способность не только извлекать факты, но и синтезировать смысл из разных мест повести.

### Пример 3. Сложный или «негативный» случай

**Вопрос**: *Есть ли в тексте прямое свидетельство, что барыня умерла в конце повести?*
**Ответ**: *Нет.*

**Анализ**:
Это провокационный вопрос, так как в некоторых пересказах можно встретить утверждение о смерти барыни. Однако в оригинальном тексте Тургенева её смерть не описана: упоминается только эпизод с недомоганием, после чего повествование сосредотачивается на судьбе Герасима. Система дала корректный ответ — в тексте нет прямого свидетельства смерти. Такой результат демонстрирует важное свойство RAG-подхода: способность не придумывать информацию, а честно фиксировать её отсутствие.